In [1]:
#default_exp typing

# Typing
> Custom types used throughout the library

In [28]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [29]:
#export
import enum
import inspect
import typing
from functools import wraps, partial

In [30]:
#export
class Member:
    """Annotation only type to be used within `@enumify`.
    
    Used to denote when a member will be of type `str`,
    with the value being its member name in lowercase.
    """
    def __init__(self, *args):
        raise TypeError("Member is a documentation type, cannot be instantiated")

Mem = Member

In [31]:
from fastcore.test import ExceptionExpected

with ExceptionExpected(TypeError, "Member is a documentation type, cannot be instantiated"):
    t = Member()

with ExceptionExpected(TypeError, "Member is a documentation type, cannot be instantiated"):
    t = Mem()

In [49]:
#export
class Documented:
    """Annotation only type to be used within `@enumify`.
    
    Used to denote when a member will have a docstring as its last assert.
    """
    def __init__(self, *args):
        raise TypeError("Documented is a documentation type, cannot be instantiated")

Doc = Documented

In [50]:
from fastcore.test import ExceptionExpected

with ExceptionExpected(TypeError, "Documented is a documentation type, cannot be instantiated"):
    t = Documented()

with ExceptionExpected(TypeError, "Documented is a documentation type, cannot be instantiated"):
    t = Doc()

In [60]:
#export
class FunctionalEnum(enum.Enum):
    """
    An `Enum` class implementing `__ne__`, `__eq__`, and `__str__` to compare `self.value`.
    
    Compatible with the functional API.
    """
    def __str__(self): return str(self.value)
    def __eq__(self, other): return getattr(other, "value", other) == self.value
    def __ne__(self, other): return getattr(other, "value", other) != self.value

In [61]:
_da = [["zero", 0], ["one", 1]]

_d = FunctionalEnum("test_enum", _da)
test_eq(hasattr(_d, "zero"), True)
test_eq(str(_d.zero), "0")
test_eq(_d.zero == 0, True)

test_eq(hasattr(_d, "one"), True)
test_eq(str(_d.one), "1")
test_eq(_d.one == 1, True)

In [62]:
#export
class DocumentedEnum(FunctionalEnum):
    """
    An `Enum` capabile of having its members have docstrings.
    
    Inherits `FunctionalEnum` to allow for logic comparison via `==`, `!=`,
    and string representation `str()` of `self.value`.
    """
    def __init__(self, *args):
        """
        Creates a generic enumeration with assigning of a member docstring

        Should be passed in the form of:
          docstring, value
        """
        if args[0] is not None:
            self.__doc__ = args[0]
        if len(args) > 1:
            self._value_ = args[1]
        else:
            self._value_ = None

In [63]:
_da = [["addition", ("Sum of two numbers", "addition")], ["subtraction", ("Some documentation")], ["multiplication", (None, "multiplication")]]

_d = DocumentedEnum("test_enum", _da)
test_eq(hasattr(_d, "addition"), True)
test_eq(str(_d.addition), "addition")
test_eq(_d.addition.__doc__, "Sum of two numbers")
test_eq(_d.addition == "addition", True)

test_eq(str(_d.subtraction), str(None))
test_eq(_d.subtraction.__doc__, "Some documentation")
test_eq(_d.subtraction != "addition", True)

test_eq(str(_d.multiplication), "multiplication")
test_eq(_d.multiplication.__doc__, "An enumeration.")
test_eq(_d.multiplication != "addition", True)

In [70]:
#export
def multiassert(type, types:list=[]):
    """Runs `==` on all `types`
    
    If any are `True`, returns `True`.
    """
    for t in types:
        if t == type:
            return True
    return False

In [75]:
#export
def _assign_annotations(cls):
    """
    Creates a `DocumentedEnum` based on annotations and asserts in `cls`
    """
    # First, filter out all but what we need: the doc, annotations, and any set members
    d = dict(cls.__dict__)
    _keep = ["__doc__", "__annotations__"]
    for key in list(d):
        if key.startswith('_') and key not in _keep:
            d.pop(key, None)
    names = [] # Names for our enum
    keys = []
    # Next get our members with out values
    for name, typ in list(d["__annotations__"].items()):
        if not multiassert(typ, [Member, Documented, tuple[Member, Documented]]):
            continue
        if typ == tuple[Member, Documented]:
            doc = getattr(cls, name)
            value = name.lower()
        elif typ == Documented:
            value, doc = getattr(cls, name)
        else: # Emplicit else for raw Member
            doc = "An enumeration."
            value = name.lower()
        names.append([name, (doc, value)])
        keys.append(name)
    
    # For any values set like a regular enum
    for name in d:
        if name not in keys and not name.startswith("_"):
            names.append([name, ("An enumeration.", getattr(cls, name))])
            keys.append(name)
    new_cls = DocumentedEnum(value=cls.__name__, names=names)
    new_cls.__doc__ = cls.__doc__
    return new_cls

In [78]:
#hide
class DaysOfWeek:
    MONDAY:tuple[Member,Documented] = "First day of the week"
    TUESDAY:Member
    WEDNESDAY:Documented = "Wed", "Third day of the week"
    THURSDAY:int = 0
    
NewAnnotation = _assign_annotations(DaysOfWeek)
test_eq(NewAnnotation.MONDAY, "monday")
test_eq(NewAnnotation.MONDAY.__doc__, "First day of the week")
test_eq(NewAnnotation.TUESDAY, "tuesday")
test_eq(NewAnnotation.TUESDAY.__doc__, "An enumeration.")
test_eq(NewAnnotation.WEDNESDAY, "Wed")
test_eq(NewAnnotation.WEDNESDAY.__doc__, "Third day of the week")
test_eq(NewAnnotation.THURSDAY, 0)
test_eq(NewAnnotation.THURSDAY.__doc__, "An enumeration.")

In [80]:
#hide
class DaysOfWeek:
    MONDAY:tuple[Mem,Doc] = "First day of the week"
    TUESDAY:Mem
    WEDNESDAY:Doc = "Wed", "Third day of the week"
    THURSDAY:int = 0
    
NewAnnotation = _assign_annotations(DaysOfWeek)
test_eq(NewAnnotation.MONDAY, "monday")
test_eq(NewAnnotation.MONDAY.__doc__, "First day of the week")
test_eq(NewAnnotation.TUESDAY, "tuesday")
test_eq(NewAnnotation.TUESDAY.__doc__, "An enumeration.")
test_eq(NewAnnotation.WEDNESDAY, "Wed")
test_eq(NewAnnotation.WEDNESDAY.__doc__, "Third day of the week")
test_eq(NewAnnotation.THURSDAY, 0)
test_eq(NewAnnotation.THURSDAY.__doc__, "An enumeration.")

## enumify

In [81]:
#export
def enumify(cls=None):
    """
    A decorator to turn `cls` into an Enum class with member values as property names, and potentially with documentation
    
    Should be documented with the `Member` type with the following annotation:
    ```python
    from fastreinference.typing import Member
    @enumify
    class MyClass:
      NAME:Member["Some documented enum value"]
      name_two:Member # An undocumented enum value
      name_three:Member["Some documentation"] = "some value"
    ```
    
    Can also use the shorthand `Mem` type
    """
    def wrap(cls): return _assign_annotations(cls)
    if cls is None:
        return partial(enumify)
    return wrap(cls)

In [83]:
@enumify
class DaysOfWeek:
    MONDAY:tuple[Member,Documented] = "First day of the week"
    TUESDAY:Member
    WEDNESDAY:Documented = "Wed", "Third day of the week"
    THURSDAY:Documented = 0, "Fourth day of the week"
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, "First day of the week")
test_eq(DaysOfWeek.TUESDAY, "tuesday")
test_eq(DaysOfWeek.TUESDAY.__doc__, "An enumeration.")
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, "Third day of the week")
test_eq(DaysOfWeek.THURSDAY, 0)
test_eq(DaysOfWeek.THURSDAY.__doc__, "Fourth day of the week")

In [84]:
@enumify
class DaysOfWeek:
    MONDAY:tuple[Mem,Doc] = "First day of the week"
    TUESDAY:Mem
    WEDNESDAY:Doc = "Wed", "Third day of the week"
    THURSDAY:Doc = 0, "Fourth day of the week"
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, "First day of the week")
test_eq(DaysOfWeek.TUESDAY, "tuesday")
test_eq(DaysOfWeek.TUESDAY.__doc__, "An enumeration.")
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, "Third day of the week")
test_eq(DaysOfWeek.THURSDAY, 0)
test_eq(DaysOfWeek.THURSDAY.__doc__, "Fourth day of the week")

In [85]:
from dataclasses import dataclass

In [90]:
@dataclass
class SomeClass:
    o:Documented = "foo"

In [92]:
dataclass?

Signature:
dataclass(
    cls=None,
    /,
    *,
    init=True,
    repr=True,
    eq=True,
    order=False,
    unsafe_hash=False,
    frozen=False,
)
Docstring:
Returns the same class as was passed in, with dunder methods
added based on the fields defined in the class.

Examines PEP 526 __annotations__ to determine fields.

If init is true, an __init__() method is added to the class. If
repr is true, a __repr__() method is added. If order is true, rich
comparison dunder methods are added. If unsafe_hash is true, a
__hash__() method function is added. If frozen is true, fields may
not be assigned to after instance creation.
File:      /opt/conda/lib/python3.9/dataclasses.py
Type:      function
